## Page Rank

My notes for Page Rank, pictures are taken from the course slides.

#### Flow Formulation

Consider the following graph

![Graph](SampleGraph.png)

The Importance of a node is the proportional to the importance of the incoming node

thus 

$r_y\:=\:\frac{r_y}{2} + \frac{r_a}{2}$

$r_a\:=\:\frac{r_y}{2} + r_m$

$r_m\:=\:\frac{r_a}{2}$

We can therefore say
$r_j\:=\:\sum_{i \to y}\frac{r_i}{d_i}$

where $d_i$ is the out degree of node i. We add additional constraint $r_y+r_m+r_a = 1$. We can use gaussian elimination to solve these three unknown with the four available equations.

#### Matrix formulation.

The above formulation can be expressed in terms of matrix. If we have a link between i and j $i \to j$, then we set $M_{ji} = \frac{1}{d_i}$ else set $M_{ji} = 0$. By doing this we ensure that the sum of values in the column add to 1. In other words, M is column stochastic.

For the matrix M, if we read values across the rows, we essentially see all the incoming edges and their weight.

Recall for all $i \to j$, we have $r_j\:=\:\sum_{i \to y}\frac{r_i}{d_i}$ We therefore can express the above formulation in matrix form as $r\:=\:M \cdot r$. Also for the above graph, the flow equations given above can be expressed as matrix as follows

$\begin{bmatrix}
    r_y \\
    r_a \\
    r_m  
\end{bmatrix}$
=
$\begin{bmatrix}
    \frac{1}{2}       & \frac{1}{2} & 0 \\
    \frac{1}{2}      & 0 & 1 \\
    0       & \frac{1}{2} & 0  
\end{bmatrix} \cdot$
$\begin{bmatrix}
    r_y \\
    r_a \\
    y_m  
\end{bmatrix}$


Lets compute the pagerank below by finding eigen vectors and values

In [1]:
from numpy import linalg as LA
import numpy as np
M = np.matrix([[0.5, 0.5, 0], [0.5, 0, 1], [0, 0.5, 0]], dtype = np.float32) #for float64 we get rounding error
ev, evec = LA.eig(M)
ranks = evec[:, ev == 1]
#ranks need to sum up to 1, normalize them
ranks /= sum(ranks)
print('Page ranks are ', ranks.flatten())

Page ranks are  [[ 0.40000001  0.40000001  0.2       ]]



#### Power Iteration Method

The following method is power iteration methos where we iterate to solve $r = M \cdot r$ 
till $\vert r^{(t)} - r^{(t + 1)} \vert < \epsilon$. Let us implement Power iteration method below

In [2]:
def pagerank_pi(M, epsilon = 0.00000001, max_iterations = 200):
    numpages = M.shape[1]
    r = np.matrix(np.ones(numpages)).reshape((-1, 1)) / numpages
    r_new = M * r
    iters = 1
    while np.max(np.abs(r_new - r)) > epsilon:
        if iters > max_iterations:
            raise Exception('Max iterations reached, algorithms not yet converged')
        r = r_new
        r_new = M * r
        iters += 1
    
    return r_new, iters


r, iters = pagerank_pi(M)
print('Page rank by power iteration is ', r.flatten(), ', found in', iters, ' iterations')

Page rank by power iteration is  [[ 0.4  0.4  0.2]] , found in 81  iterations


#### Random Walk Interpretation

Suppose a surfer is on node i of the graph at time t, the surfer can go to node j which can be reached from the outlinks of node i at time t + 1 and the process repeats indefinitely.

Let P(t) be a vector whose $i^{th}$ element denote the probability we are on node i at time t. Thus p(t) is the probability distribution over pages in the graph.

$p(t +1) = M \cdot p(t)$

when $p(t +1) = M \cdot p(t) = p(t)$, random walk reaches a stationary state.

#### Problems

##### Spider trap Problem

Consider the following graph

![SpiderTrap](SpiderTrap.png)

We will recursively loop betweem the two and never converge.

##### Dead End problem.

The following problem represents the Deadend problem

![DeadEnd](DeadEnd.png)

The problem ends with imbalanced probablity distribution with both weights zeroed out ( weights leak out).

In [3]:
#Spider trap problem
M = np.matrix([[0.5, 0.5, 0], [0.5, 0, 0], [0, 0.5, 1]], dtype = np.float32)
print('M is\n', M)
rank, _ = pagerank_pi(M)
print('\nRank for graph with spider trap is ', rank.flatten())

#Dead end problem
M = np.matrix([[0.5, 0.5, 0], [0.5, 0, 0], [0, 0.5, 0]], dtype = np.float32)
print('\nM is\n', M)
rank, _ = pagerank_pi(M)
print('\nRank for graph with dead end is ', rank.flatten())

M is
 [[ 0.5  0.5  0. ]
 [ 0.5  0.   0. ]
 [ 0.   0.5  1. ]]

Rank for graph with spider trap is  [[  2.58264849e-08   1.59616455e-08   9.99999958e-01]]

M is
 [[ 0.5  0.5  0. ]
 [ 0.5  0.   0. ]
 [ 0.   0.5  0. ]]

Rank for graph with dead end is  [[  3.94593577e-08   2.43872243e-08   1.50721335e-08]]



As we see above the node with a self loop and no other outgoing edge got weight of all nodes other than itself  reduced to nearly 0 (reducing to a value less then epsilon).

On other hand, dead ends make the matrix non column stochastic as there those columns dont add up to 1. Also the weights leak out reducing the weights of all the nodex to 0 (reducing to a value less then epsilon).

A solution to dead ends is to teleport. Teleporting ensures that the matrix is column stochastic and ensures when we reach a dead end node, we jump to any node in the graph (including itself) with equal probability. To below is the implementation of power iteration with teleporting

In [4]:
def pagerank_with_teleport(M, epsilon = 0.00000001, max_iterations = 200):
    nodes = M.shape[1]
    e = np.matrix(np.ones((1, nodes))) / nodes
    #Ones for columns whose sum adds to 0
    a = 1 - np.sum(M, axis = 0)
    return pagerank_pi(M + e.T * a, epsilon, max_iterations)
    

In [5]:
#Dead end problem
M = np.matrix([[0.5, 0.5, 0], [0.5, 0, 0], [0, 0.5, 0]], dtype = np.float32)
print('\nM is\n', M)
rank, _ = pagerank_with_teleport(M)
print('\nRank for graph with dead end is ', rank.flatten())


M is
 [[ 0.5  0.5  0. ]
 [ 0.5  0.   0. ]
 [ 0.   0.5  0. ]]

Rank for graph with dead end is  [[ 0.46153846  0.30769231  0.23076923]]



As seen above, the dead end node didnt drive down all the weights to 0 and appropriate rank givem to all the pages with the node with dead end given least weight.

#### How Teleports solve the problem

$r^{(t + 1)} = M \cdot r^{(t)}$

##### Markov Chains are

- X is a set of states
- Let P be the transition probability such that $P_{ij} = P(X_{t} = i \vert X_{t - 1} = j)$
- $\pi$ is the stationary probability of being in state x $\in$ X
- We need to find $\pi$ sich that $\pi = P \cdot \pi$

The theory of Markov chains state that for any vector power iteration will converge to a positive stationary vector if P is stochastic, irreducible and aperiodic

For a matrix to be stochastic we ensure that the sum of all its columns is 1. If not, we ensure that the value is set as given in ``pagerank_with_teleport``

A matrix is periodic if the time taken for two consecutive visits to a node is always a multiple of k (k > 1). Following is an example of periodic graph (with onlt black edges) and adding the green edges makes it aperiodic

![MakeAperiodic](MakeAPeriodic.png)

A matrix is irreducible if there is a non zero probability of going from any state to another. The following image shows an irreducible graph. The original graph is with black edges only, adding green edges makes it irreducible

![Irreducible](Irreducible.png)


The Google formulation of Page rank

$r_j\:=\:\sum_{i \to j}\beta \frac{r_i}{d_j} + (1 - \beta)\frac{1}{n}$

Note that the above formulation assumes there are no dead ends. The solution for this to fix for dead ends is to add another term to ensure we teleport to another node with probability 1 for dead end nodes. Following implementation use the Google's page rank method, with the fix for dead ends and then use power iteration to get the solution as the matrix is stochastic, aperiodic and irreducible

In [6]:
def google_pagerank(M, beta = 0.8, epsilon = 0.00000001, max_iterations = 200):
    dim = M.shape[1]
    e = np.matrix(np.ones((dim, 1)))
    dead_ends = 1 - np.sum(M, axis = 0)
    #The additional term "(beta * np.matrix(np.ones((dim, 1))) / dim) * dead_ends" ensures that the 
    #dead ends dont cause the algorithm to fail
    M_hat = beta * M + (1 - beta) * e * e.T / dim  + (beta * np.matrix(np.ones((dim, 1))) / dim) * dead_ends
    return pagerank_pi(M_hat, epsilon, max_iterations)

In [8]:
M = np.matrix([[0.5, 0.5, 0], [0.5, 0, 0], [0, 0.5, 1]], dtype = np.float32)
rank, _ = google_pagerank(M)
print('Pagerank for matrix with spider trap is', rank.flatten())

#With dead end
M = np.matrix([[0.5, 0.5, 0], [0.5, 0, 0], [0, 0.5, 0]], dtype = np.float32)
rank, _ = google_pagerank(M)
print('Pagerank with dead ends is', rank.flatten())


Pagerank for matrix with spider trap is [[ 0.21212122  0.15151516  0.63636362]]
Pagerank with dead ends is [[ 0.43209877  0.30864197  0.25925926]]




TODO: See "How we really compute page rank" section